In [157]:
from __future__ import division
from sklearn.preprocessing import CategoricalEncoder
from sklearn.preprocessing import LabelEncoder

import torchvision.models as models
from multiprocessing import Pool
import time
import numpy as np
import pandas as pd
import datetime as dt
import os.path as dpath
import matplotlib.pyplot as plt
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from sklearn.model_selection import train_test_split
from mpl_toolkits.axes_grid1 import ImageGrid
from PIL import Image
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models
import pdb
from pathlib import Path
import torch.optim as optim


import torch.nn as nn
import torch.nn.functional as F

from IPython.display import FileLink, FileLinks


%matplotlib inline

In [2]:
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)


In [3]:
path = "/home/edwin/Datasets/competitions/avito-demand-prediction/"
PATH = Path(path)
TRN_CSV = pd.read_csv(PATH/'train.csv')
train, valid = train_test_split(TRN_CSV, train_size=300, test_size=10)
sample = train.copy()

In [7]:
len(train)

300

In [ ]:
?? ColumnarModelData

In [ ]:
?? np.stack

In [ ]:
# create an array for every column

In [ ]:
[c.values for n, c in train.items()]

In [ ]:
columns = [c.values for n, c in train.items()]

In [ ]:
np.stack(columns, 1).shape

In [ ]:
proc_df(train, 'deal_probability', do_scale=True)

In [ ]:
df_train_copy = train.copy()

In [ ]:
df_train_copy

In [ ]:
train.user_type.astype('category').cat.as_ordered()

In [ ]:
cat_vars = []
contin_vars = []

In [ ]:
df_train_copy.sample()

In [ ]:
cat_vars = ['item_id', 'user_id', 'region', 'city', 'parent_category_name', 'category_name',
           'param_1', 'param_2', 'param_3', 'title', 'description', 'user_type','image']

In [ ]:
contin_vars = ['price', 'item_seq_number', 'image_top_1', 'deal_probability']

In [ ]:
for v in cat_vars:
    df_train_copy[v] = df_train_copy[v].astype('category').cat.as_ordered() 

In [ ]:
for v in contin_vars:
    df_train_copy[v] = df_train_copy[v].astype('float32')

In [ ]:
proc_df(df_train_copy, 'deal_probability', do_scale=True)

In [ ]:
df_train_copy = df_train_copy.drop(['activation_date', 'image'], axis=1)

In [ ]:
proc_df(df_train_copy, 'deal_probability', do_scale=True)

In [ ]:
# getting categories
df_train_copy.user_type.cat.categories[0]

In [ ]:
# one hot encoding
df_train_copy.user_type.cat.codes

In [ ]:
cols = [c.values for n, c in df_train_copy.items()]

In [ ]:
cols_stack = np.stack(cols,1)

In [ ]:
cols_stack[0]

In [ ]:
# Turn Category Variables into Cats
# Create Y

In [ ]:
TRN_CSV

In [ ]:
TRN_CSV_COPY = TRN_CSV.copy()

In [ ]:
TRN_CSV_COPY.columns

In [ ]:
TRN_CSV_COPY.sample()

In [ ]:
CSV = TRN_CSV
CSV_COPY = CSV.copy()
cat_vars = ['item_id', 'user_id', 'region', 'city', 'parent_category_name', 
            'category_name', 'param_1', 'param_2', 'param_3', 'title', 'description',
            'user_type']
contin_vars = ['price', 'item_seq_number', 'image_top_1']
drop_vars = ['activation_date', 'image']
y_vars = ['deal_probability']
for v in cat_vars:
    CSV_COPY[v] = CSV_COPY[v].astype('category').cat.as_ordered()
for v in contin_vars:
    CSV_COPY[v] = CSV_COPY[v].astype('float32')

In [ ]:
sample = CSV_COPY[:100]

In [ ]:
for column in sample:
    if sample.dtypes[column] == 'category':
        print(column)


In [ ]:
for column in sample:
    if hasattr(sample[column], 'cat'):
        sample[column] = sample[column].cat.codes

In [ ]:
sample.drop()

In [ ]:
sample = sample.drop(['activation_date','image'], axis=1)

In [ ]:
for v in sample:
    sample[v] = sample[v].astype('float32')

In [ ]:
cols_stack = [c.values for n, c in sample.items()]

In [ ]:
data = np.stack(cols_stack, 1)

In [ ]:
y = data[:, 15]

In [ ]:
y

In [ ]:
data = np.delete(data, 15, 1)

In [ ]:
data

In [ ]:
# delete the last column

In [ ]:
np.delete(data, 15, 1).shape

In [ ]:
data[:,15]

In [ ]:
list(sample.items())[0]

In [ ]:
?? proc_df

In [ ]:
class AvitoDataset(Dataset):
    def __init__(self, CSV):
        return ""
    
    def __len__(self):
        return len(self.y)

In [ ]:
TRN_CSV_COPY

In [ ]:
data.shape

In [ ]:
y.shape

In [ ]:
class AvitoDataset(Dataset):
    def __init__(self):
        self.data = data
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return [self.data[idx], self.y[idx]]

In [ ]:
train_ds = AvitoDataset()

In [ ]:
train_ds[0]

In [ ]:
data

In [ ]:
train_dl = DataLoader(train_ds, batch_size=64, shuffle=False, num_workers=4)

In [ ]:
x, y = next(iter(train_dl))

In [ ]:
x.shape, y.shape

In [97]:
class AvitoModel(nn.Module):
    def __init__(self):
        super(AvitoModel, self).__init__()
        self.fc1 = nn.Linear(1, 1)
        
    def forward(self, x):
        out = self.fc1(x)
        return out

In [98]:
model = AvitoModel()
model.cuda()

AvitoModel(
  (fc1): Linear(in_features=1, out_features=1, bias=True)
)

In [99]:
criterion = nn.MSELoss()
learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [100]:
num_epochs = 1000

In [33]:
sample = sample.fillna(0)

In [128]:
x = sample.price.values
y = sample.deal_probability.values

In [160]:
from sklearn.preprocessing import normalize

In [ ]:
normalize(X, norm=’l2’,

In [166]:
y_train

array([[ 0.09564],
       [ 0.2    ],
       [ 0.2    ],
       [ 0.     ],
       [ 0.     ],
       [ 0.05499],
       [ 0.13431],
       [ 0.     ],
       [ 0.20406],
       [ 0.     ],
       [ 0.     ],
       [ 0.     ],
       [ 0.78503],
       [ 0.     ],
       [ 0.     ],
       [ 0.19917],
       [ 0.     ],
       [ 0.7376 ],
       [ 0.     ],
       [ 0.61754],
       ..., 
       [ 0.12981],
       [ 0.     ],
       [ 0.     ],
       [ 0.     ],
       [ 0.     ],
       [ 0.86521],
       [ 0.4    ],
       [ 0.09634],
       [ 0.18706],
       [ 0.80323],
       [ 0.12152],
       [ 0.     ],
       [ 0.     ],
       [ 0.76786],
       [ 0.02444],
       [ 0.80323],
       [ 0.86521],
       [ 0.78503],
       [ 0.73757],
       [ 0.     ]], dtype=float32)

In [164]:
normalize(x_train, norm='l2')

array([[ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       ..., 
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.]], dtype=float32)

In [154]:
x_train = np.array(x_train, dtype=np.float32)
y_train = np.array(y, dtype=np.float32)

x_train = x_train.reshape(-1,1)
y_train = y_train.reshape(-1,1)

In [155]:
num_epochs = 300

In [156]:
iter = 0
for epoch in range(num_epochs):
    x_ = Variable(torch.from_numpy(x_train).cuda())
    y_ = Variable(torch.from_numpy(y_train).cuda())
    
    optimizer.zero_grad()
        
    outputs = model(x_)
        
    loss = criterion(outputs, y_)
    
    loss.backward()
        
    optimizer.step()
    iter += 1
        
    print('Iteration: {}. Loss: {}.'.format(iter, loss.data[0]))

Iteration: 1. Loss: nan.
Iteration: 2. Loss: nan.
Iteration: 3. Loss: nan.
Iteration: 4. Loss: nan.
Iteration: 5. Loss: nan.
Iteration: 6. Loss: nan.
Iteration: 7. Loss: nan.
Iteration: 8. Loss: nan.
Iteration: 9. Loss: nan.
Iteration: 10. Loss: nan.
Iteration: 11. Loss: nan.
Iteration: 12. Loss: nan.
Iteration: 13. Loss: nan.
Iteration: 14. Loss: nan.
Iteration: 15. Loss: nan.
Iteration: 16. Loss: nan.
Iteration: 17. Loss: nan.
Iteration: 18. Loss: nan.
Iteration: 19. Loss: nan.
Iteration: 20. Loss: nan.
Iteration: 21. Loss: nan.
Iteration: 22. Loss: nan.
Iteration: 23. Loss: nan.
Iteration: 24. Loss: nan.
Iteration: 25. Loss: nan.
Iteration: 26. Loss: nan.
Iteration: 27. Loss: nan.
Iteration: 28. Loss: nan.
Iteration: 29. Loss: nan.
Iteration: 30. Loss: nan.
Iteration: 31. Loss: nan.
Iteration: 32. Loss: nan.
Iteration: 33. Loss: nan.
Iteration: 34. Loss: nan.
Iteration: 35. Loss: nan.
Iteration: 36. Loss: nan.
Iteration: 37. Loss: nan.
Iteration: 38. Loss: nan.
Iteration: 39. Loss: 

In [141]:
import numpy as np
from sklearn.preprocessing import normalize

In [149]:
x_train = x/x.max()

In [144]:
normalize(x,norm='l2')

ValueError: Expected 2D array, got 1D array instead:
array=[     4000.       500.         0.       600.      3700.   1300000.     50000.      1500.      1700.     22000.
       900.       200.       500.         0.       380.   1900000.      1500.    330000.       500.         0.
 ...,   1000000.      2000.       700.      2100.       100.      9300.      1000.    650000.         0.
      1000.       300.       200.      3000.     10000.  41280000.       150.       800.      2000.    268000.
       500.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.